In [1]:
import model_utils as mdu
import numpy as np
import pandas as pd

In [2]:
def predictor(model,data):
    data['year']=data.datetime.apply(lambda x:x.split('-')[0])
    data['month']=data.datetime.apply(lambda x:x.split('-')[1])
    data['date']=data.datetime.apply(lambda x:x.split('-')[2].split()[0])
    data['time']=data.datetime.apply(lambda x:x.split()[1].split(':')[0])
    
    X_date=data.iloc[:,-4:]
    
    data=data.drop(X_date,axis=1)
    data=data.drop(['datetime'],axis=1)
    
    data=X_date.join(data)
    
    X_season=data['season'].values
    X_weather=data['weather'].values
    
    data=data.drop(['season','weather'],axis=1)
    
    X_season=X_season.reshape(len(data),1)
    X_weather=X_weather.reshape(len(data),1)
    
    X_season=model['ohe1'].transform(X_season).toarray().astype(int)
    X_weather=model['ohe2'].transform(X_weather).toarray().astype(int)
    X_season=X_season[:,1:]
    X_weather=X_weather[:,1:]
    
    X_sw=np.append(X_season,X_weather,axis=1)
    
    sw=pd.DataFrame(X_sw)
    X=data.join(sw,how='right')
    X=X.values
    
    X=model['scaler'].transform(X)
    
    result=model['model'].predict(X).astype(int)
    return result

In [3]:
datetime=input('Datetime: ')
season=int(input('Season(1-4): '))
holiday=int(input('Holiday(0 or 1): '))
workingday=int(input('Working Day(0 or 1): '))
weather=int(input('Weather(1-4): '))
temp=float(input('Temperature: '))
atemp=float(input('Temperature feels like: '))
humidity=int(input('Humidity: '))
windspeed=float(input('Windspeed: '))

data={
    'datetime':datetime,
    'season':season,
    'holiday':holiday,
    'workingday':workingday,
    'weather':weather,
    'temp':temp,
    'atemp':atemp,
    'humidity':humidity,
    'windspeed':windspeed
}

data=pd.DataFrame([data])

ValueError: invalid literal for int() with base 10: ''

In [28]:
ohe1=mdu.load_model(file='bike_sharing_ohe1.p')
ohe2=mdu.load_model(file='bike_sharing_ohe2.p')
scaler=mdu.load_model(file='bike_sharing_scaler.p')
regressor=mdu.load_model(file='bike_sharing_model.p')

model={
    'ohe1':ohe1,
    'ohe2':ohe2,
    'scaler':scaler,
    'model':regressor
}

In [29]:
test_data=pd.read_csv('input/bike-sharing-demand/test.csv')

In [30]:
test_pred=predictor(model,test_data)

In [31]:
submission=pd.DataFrame(test_pred,index=test_data['datetime'],columns=['Count Predictions'])

In [32]:
submission.to_csv(r'output/submission.csv')